In [1]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'chatbot'
device = 'cuda'

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence.lower())

In [3]:
tokenize('$100')

['$', '100']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(words))
    for idx,w in enumerate(words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [8]:
# data = pd.read_csv('./data.csv').sample(frac=1)[:3250]

In [9]:
# data.to_csv('./cleaned-data.csv',index=False)

In [10]:
data = pd.read_json('./data.json')

In [11]:
dict(data.iloc[0]['intents'])

{'tag': 'greeting',
 'patterns': ['Hi',
  'Hey',
  'How are you',
  'Is anyone there?',
  'Hello',
  'Good day'],
 'responses': ['Hey :-)',
  'Hello, thanks for visiting',
  'Hi there, what can I do for you?',
  'Hi there, how can I help?']}

In [12]:
new_data = {'Question':[],'Tag':[]}
responses = {'Tag':[],'Response':[]}

In [13]:
for i in range(len(data)):
    info = data.iloc[i]['intents']
    for q in list(info['patterns']):
        new_data['Tag'].append(info['tag'])
        new_data['Question'].append(q)
    for r in list(info['responses']):
        responses['Tag'].append(info['tag'])
        responses['Response'].append(r)

In [14]:
data = pd.DataFrame(new_data)

In [15]:
data.to_csv('./data.csv')

In [16]:
responses = pd.DataFrame(responses)

In [17]:
X = data['Question']
y = data['Tag']
X_words = []
labels = {}
labels_r = {}
idx = 0
data = []

In [18]:
for label in y:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [19]:
labels

{'greeting': 1,
 'goodbye': 2,
 'thanks': 3,
 'items': 4,
 'payments': 5,
 'delivery': 6,
 'funny': 7}

In [20]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    X_words.extend(new_X)
    
    data.append([
        X_words,
        labels[y_batch]-1
    ])

In [21]:
X_words = sorted(set(X_words))
np.random.shuffle(data)

In [22]:
X = []
y = []

In [23]:
for input_sentence,output_sentence in tqdm(data):
    X.append(bag_of_words(input_sentence,X_words))
    y.append(output_sentence)

In [24]:
X_train = torch.from_numpy(np.array(X)).to(device).long()
y_train = torch.from_numpy(np.array(y)).to(device).long()
X_test = torch.from_numpy(np.array(X)).to(device).long()
y_test = torch.from_numpy(np.array(X)).to(device).long()

In [25]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [26]:
def get_accuracy(model,X,y,):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [27]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.dropout = Dropout()
        self.hidden = 128
        self.batchnorm = BatchNorm1d(self.hidden)
        self.linear1 = Linear(len(X_words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear3 = Linear(self.hidden,self.hidden)
        self.linear4 = Linear(self.hidden,self.hidden)
        self.linear5 = Linear(self.hidden,self.hidden)
        self.output = Linear(self.hidden,len(labels)) # TODO trying adding 1 or -1
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.activation(self.linear2(preds))
        preds = self.batchnorm(self.dropout(self.activation(self.linear3(preds))))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output(preds)
        return preds

In [28]:
model = Model().to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 1000
batch_size = 8

In [29]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    preds = model(X_train.float())
    loss = criterion(preds,y_train.long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    torch.cuda.empty_cache()
    wandb.log({'Loss':get_loss(model,X_train.float(),y_train,criterion)})
    torch.cuda.empty_cache()
#     wandb.log({'Val Loss':get_loss(model,X_test.float(),y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':get_accuracy(model,X_train.float(),y_train)})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test.float(),y_test)})
    torch.cuda.empty_cache()
wandb.finish()
torch.cuda.empty_cache()

In [30]:
torch.save(model.to('cpu'),'model.pt')
torch.save(model.to('cpu'),'model.pth')
torch.save(model.to('cpu').state_dict(),'model-sd.pt')
torch.save(model.to('cpu').state_dict(),'model-sd.pth')
torch.save(X_words,'words.pt')
torch.save(X_words,'words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')
torch.save(labels_r,'labels_r.pt')
torch.save(labels_r,'labels_r.pth')
torch.save(idx,'idx.pt')
torch.save(idx,'idx.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')
torch.save(y,'y.pt')
torch.save(y,'y.pth')

In [31]:
model.eval()
# torch.argmax(model(torch.from_numpy(bag_of_words(['hi','how','are','you'],X_words)).float()))

Model(
  (activation): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=56, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=128, bias=True)
  (linear4): Linear(in_features=128, out_features=128, bias=True)
  (linear5): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=7, bias=True)
)

In [32]:
responses

         Tag                                           Response
0   greeting                                            Hey :-)
1   greeting                         Hello, thanks for visiting
2   greeting                   Hi there, what can I do for you?
3   greeting                          Hi there, how can I help?
4    goodbye                 See you later, thanks for visiting
5    goodbye                                    Have a nice day
6    goodbye                         Bye! Come back again soon.
7     thanks                                     Happy to help!
8     thanks                                          Any time!
9     thanks                                        My pleasure
10     items                             We sell coffee and tea
11     items                             We have coffee and tea
12  payments              We accept VISA, Mastercard and Paypal
13  payments      We accept most major credit cards, and Paypal
14  delivery                            

In [33]:
model.train()

Model(
  (activation): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=56, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=128, bias=True)
  (linear4): Linear(in_features=128, out_features=128, bias=True)
  (linear5): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=7, bias=True)
)

In [34]:
X_train[0].shape

torch.Size([56])

In [35]:
preds = model(X_train.cpu().float()) #[25].view(1,-1).to('cpu').float().to('cpu')

In [36]:
preds

tensor([[ 3.4844e-01, -2.5719e-01, -7.3051e-02, -3.1737e-01, -1.0198e-02,
         -1.8253e-01, -3.0912e-01],
        [ 4.8260e-01, -2.4179e-01, -7.2495e-04, -2.1310e-01,  5.9737e-02,
         -3.2040e-01, -3.2879e-01],
        [ 3.2142e-01, -3.2436e-01,  6.2441e-02, -2.7595e-01, -5.0785e-02,
         -1.7488e-01, -2.0841e-01],
        [ 3.9503e-01, -2.7434e-01,  3.1600e-02, -3.3312e-01, -1.5531e-02,
         -2.1787e-01, -2.3595e-01],
        [ 4.7485e-01, -2.7206e-01, -2.3484e-02, -1.7994e-01,  6.1659e-02,
         -3.0972e-01, -3.6239e-01],
        [ 3.9284e-01, -3.5354e-01,  3.1928e-03, -2.7831e-01,  5.2990e-02,
         -3.0946e-01, -2.5906e-01],
        [ 4.3730e-01, -2.8428e-01,  6.9800e-03, -2.9277e-01, -2.1718e-02,
         -3.1814e-01, -2.8361e-01],
        [ 2.9711e-01, -2.2302e-01, -2.3948e-02, -2.3033e-01,  4.8522e-02,
         -3.0938e-01, -2.4710e-01],
        [ 3.7027e-01, -2.4100e-01,  1.7648e-02, -2.2498e-01,  4.8779e-02,
         -3.1303e-01, -2.5069e-01],
        [ 

In [37]:
preds[0]

tensor([ 0.3484, -0.2572, -0.0731, -0.3174, -0.0102, -0.1825, -0.3091],
       grad_fn=<SelectBackward>)

In [38]:
Softmax()(preds[4])

tensor([0.2409, 0.1141, 0.1463, 0.1251, 0.1593, 0.1099, 0.1043],
       grad_fn=<SoftmaxBackward>)

In [39]:
model.eval()

Model(
  (activation): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=56, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=128, bias=True)
  (linear4): Linear(in_features=128, out_features=128, bias=True)
  (linear5): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=7, bias=True)
)

In [40]:
preds = model(X_train.cpu().float()) #[25].view(1,-1).to('cpu').float().to('cpu')

In [41]:
preds

tensor([[ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.0029, -0.2576, -0.1935],
        [ 0.2834, -0.2280, -0.0242, -0.2369,  0.002

In [42]:
torch.argmax(preds)

tensor(0)

In [43]:
responses.to_csv('./responses.csv')

In [44]:
model.eval()
# torch.argmax(model(torch.from_numpy(bag_of_words(['hi','how','are','you'],X_words)).float()))

Model(
  (activation): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=56, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=128, bias=True)
  (linear4): Linear(in_features=128, out_features=128, bias=True)
  (linear5): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=7, bias=True)
)

In [45]:
responses

         Tag                                           Response
0   greeting                                            Hey :-)
1   greeting                         Hello, thanks for visiting
2   greeting                   Hi there, what can I do for you?
3   greeting                          Hi there, how can I help?
4    goodbye                 See you later, thanks for visiting
5    goodbye                                    Have a nice day
6    goodbye                         Bye! Come back again soon.
7     thanks                                     Happy to help!
8     thanks                                          Any time!
9     thanks                                        My pleasure
10     items                             We sell coffee and tea
11     items                             We have coffee and tea
12  payments              We accept VISA, Mastercard and Paypal
13  payments      We accept most major credit cards, and Paypal
14  delivery                            

In [46]:
model.train()

Model(
  (activation): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=56, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=128, bias=True)
  (linear4): Linear(in_features=128, out_features=128, bias=True)
  (linear5): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=7, bias=True)
)

In [47]:
X_train[0].shape

torch.Size([56])

In [48]:
preds = model(X_train.cpu().float()) #[25].view(1,-1).to('cpu').float().to('cpu')

In [49]:
preds

tensor([[ 0.3307, -0.2583, -0.0169, -0.2309, -0.0019, -0.2654, -0.2638],
        [ 0.4278, -0.2856,  0.0091, -0.2461,  0.0768, -0.2515, -0.2435],
        [ 0.2698, -0.2267,  0.0142, -0.2156, -0.0804, -0.1857, -0.1777],
        [ 0.5291, -0.3417,  0.0733, -0.3956,  0.0467, -0.3468, -0.3391],
        [ 0.4006, -0.2830,  0.0007, -0.2937,  0.0303, -0.2806, -0.3022],
        [ 0.3796, -0.2902,  0.0060, -0.3030,  0.0219, -0.3510, -0.2981],
        [ 0.4149, -0.3051, -0.0726, -0.3385,  0.0372, -0.3098, -0.3357],
        [ 0.5057, -0.3564,  0.0696, -0.2820,  0.1296, -0.3320, -0.2559],
        [ 0.3724, -0.2638, -0.0237, -0.2574, -0.0662, -0.1838, -0.3327],
        [ 0.3600, -0.2183, -0.0184, -0.2400,  0.0450, -0.1841, -0.2161],
        [ 0.3482, -0.2529, -0.0008, -0.2665, -0.0036, -0.2916, -0.2696],
        [ 0.4118, -0.2226,  0.0664, -0.2617,  0.0082, -0.3062, -0.2651],
        [ 0.4933, -0.2595,  0.0685, -0.2753,  0.0023, -0.3517, -0.2620],
        [ 0.4283, -0.3210, -0.0366, -0.2967,  0.106

In [50]:
preds[0]

tensor([ 0.3307, -0.2583, -0.0169, -0.2309, -0.0019, -0.2654, -0.2638],
       grad_fn=<SelectBackward>)

In [51]:
Softmax()(preds[4])

tensor([0.2290, 0.1156, 0.1535, 0.1144, 0.1582, 0.1159, 0.1134],
       grad_fn=<SoftmaxBackward>)

In [52]:
model.eval()

Model(
  (activation): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=56, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=128, bias=True)
  (linear4): Linear(in_features=128, out_features=128, bias=True)
  (linear5): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=7, bias=True)
)

In [53]:
preds = model(X_train.cpu().float()) #[25].view(1,-1).to('cpu').float().to('cpu')

In [54]:
preds

tensor([[ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.0016, -0.2568, -0.1927],
        [ 0.2835, -0.2268, -0.0235, -0.2362,  0.001

In [55]:
torch.argmax(preds)

tensor(0)

In [56]:
responses.to_csv('./responses.csv')

In [57]:
torch.argmax(Softmax()(preds[4]))

tensor(0)

In [58]:
torch.argmax(Softmax()(preds[5]))

tensor(0)

In [59]:
torch.argmax(Softmax()(preds[6]))

tensor(0)

In [60]:
torch.argmax(Softmax()(preds[7]))

tensor(0)

In [61]:
torch.argmax(Softmax()(preds[8]))

tensor(0)

In [62]:
torch.argmax(Softmax()(preds[9]))

tensor(0)

In [63]:
torch.argmax(Softmax()(preds[8]))

tensor(0)

In [64]:
torch.argmax(Softmax()(preds[4]))

tensor(0)

In [65]:
torch.argmax(Softmax()(preds[2]))

tensor(0)

In [66]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 8
        self.batchnorm = BatchNorm1d(self.hidden)
        self.linear1 = Linear(len(X_words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear3 = Linear(self.hidden,self.hidden)
        self.linear4 = Linear(self.hidden,self.hidden)
        self.linear5 = Linear(self.hidden,self.hidden)
        self.output = Linear(self.hidden,len(labels)) # TODO trying adding 1 or -1
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output(preds)
        return preds

In [67]:
model = Model().to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 1000
batch_size = 8

In [68]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    preds = model(X_train.float())
    loss = criterion(preds,y_train.long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    torch.cuda.empty_cache()
    wandb.log({'Loss':get_loss(model,X_train.float(),y_train,criterion)})
    torch.cuda.empty_cache()
#     wandb.log({'Val Loss':get_loss(model,X_test.float(),y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':get_accuracy(model,X_train.float(),y_train)})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test.float(),y_test)})
    torch.cuda.empty_cache()
wandb.finish()
torch.cuda.empty_cache()

In [69]:
responses.to_csv('./responses.csv')

In [70]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 8
        self.activation = ReLU()
        self.batchnorm = BatchNorm1d(self.hidden)
        self.linear1 = Linear(len(X_words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear3 = Linear(self.hidden,self.hidden)
        self.linear4 = Linear(self.hidden,self.hidden)
        self.linear5 = Linear(self.hidden,self.hidden)
        self.output = Linear(self.hidden,len(labels)) # TODO trying adding 1 or -1
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output(preds)
        return preds

In [71]:
model = Model().to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 1000
batch_size = 8

In [72]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    preds = model(X_train.float())
    loss = criterion(preds,y_train.long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    torch.cuda.empty_cache()
    wandb.log({'Loss':get_loss(model,X_train.float(),y_train,criterion)})
    torch.cuda.empty_cache()
#     wandb.log({'Val Loss':get_loss(model,X_test.float(),y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':get_accuracy(model,X_train.float(),y_train)})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test.float(),y_test)})
    torch.cuda.empty_cache()
wandb.finish()
torch.cuda.empty_cache()